In [1]:
from matplotlib import image
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv
load_dotenv()
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, os.getenv('root_path'))

In [3]:
annotations = pd.read_csv(
    os.path.join(
        os.getenv('data_path'),'datasets/vindr_ds','finding_annotations.csv'
    )
)

In [4]:
annotations.head()

,study_id,series_id,image_id,laterality,view_position,height,width,breast_birads,breast_density,finding_categories,finding_birads,xmin,ymin,xmax,ymax,split
0,48575a27b7c992427041a82fa750d3fa,26de4993fa6b8ae50a91c8baf49b92b0,4e3a578fe535ea4f5258d3f7f4419db8,R,CC,3518,2800,BI-RADS 4,DENSITY C,['Mass'],BI-RADS 4,2355.139893,1731.640015,2482.979980,1852.750000,training
1,48575a27b7c992427041a82fa750d3fa,26de4993fa6b8ae50a91c8baf49b92b0,dac39351b0f3a8c670b7f8dc88029364,R,MLO,3518,2800,BI-RADS 4,DENSITY C,['Mass'],BI-RADS 4,2386.679932,1240.609985,2501.800049,1354.040039,training
2,75e8e48933289d70b407379a564f8594,853b70e7e6f39133497909d9ca4c756d,c83f780904f25eacb44e9030f32c66e1,R,CC,3518,2800,BI-RADS 3,DENSITY C,['Global Asymmetry'],BI-RADS 3,2279.179932,1166.510010,2704.439941,2184.260010,training
3,75e8e48933289d70b407379a564f8594,853b70e7e6f39133497909d9ca4c756d,893528bc38a0362928a89364f1b692fd,R,MLO,3518,2800,BI-RADS 3,DENSITY C,['Global Asymmetry'],BI-RADS 3,1954.270020,1443.640015,2589.760010,2193.810059,training
4,c3487424fee1bdd4515b72dc3fd69813,77619c914263eae44e9099f1ce07192c,318264c881bf12f2c1efe5f93920cc37,R,CC,3518,2800,BI-RADS 4,DENSITY C,['Architectural Distortion'],BI-RADS 4,2172.300049,1967.410034,2388.699951,2147.159912,training


In [14]:
annotations.columns

Index(['study_id', 'series_id', 'image_id', 'laterality', 'view_position',
       'height', 'width', 'breast_birads', 'breast_density',
       'finding_categories', 'finding_birads', 'xmin', 'ymin', 'xmax', 'ymax',
       'split'],
      dtype='object')

In [15]:
annotations_photos = annotations[['study_id', 'image_id', 'breast_density', 'split']].drop_duplicates(subset = 'image_id')

In [16]:
annotations_photos.groupby('split').breast_density.value_counts()

split     breast_density
test      DENSITY C          3060
          DENSITY D           540
          DENSITY B           380
          DENSITY A            20
training  DENSITY C         12232
          DENSITY D          2160
          DENSITY B          1528
          DENSITY A            80
Name: breast_density, dtype: int64

In [20]:
annotations_photos.shape

(20000, 4)

In [17]:
annotations_photos.groupby('split').study_id.value_counts()

split     study_id                        
test      0025a5dc99fd5c742026f0b2b030d3e9    4
          00568c6b2c47f99e0156c9bca84c3963    4
          00dfcde5aaf6cd0aab3c3a0435632b3f    4
          00f826160094668f09b06ec671efe0ab    4
          01278b9ca2c9a45dac2d38b219c2ad76    4
                                             ..
training  ff84823721d5396ddf0558a3c95e659d    4
          ff9533c9faaebcd584bc97b8b6b44b7b    4
          ff9e4f3aa02c617fd62e2ec6782b6246    4
          ffe7a45f8390f242db3b843762a4a7aa    4
          fff2339ea4b5d2f1792672ba7d52b318    4
Name: study_id, Length: 5000, dtype: int64

In [18]:
# Check for unique study id in each subset
unique_studies = annotations_photos.groupby('split').study_id.unique()


In [19]:
np.sum(np.isin(unique_studies.test, unique_studies.training))

0

# Compare tensors

In [4]:
study_id = '0a83f94db524266b567882c2695a4cfe'

In [5]:
image_ids = os.listdir(
    os.path.join(
        os.getenv('data_path'),'datasets/vindr_ds','images',study_id
    )
)

In [24]:
image_ids

['e2af0315474e547de5863a4468bb347d.npy',
 'a995ccf924881443381b6833b8828607.npy',
 'c7bcbb76529f8aa1d75cd72ffd7357e2.npy',
 'e2af0315474e547de5863a4468bb347d_v2.npy',
 '32325a132f79168bb761f3ae2a7c9969.npy']

In [7]:
img1_npy = np.load(
        os.path.join(
            os.getenv('data_path'),'datasets/vindr_ds','images',study_id, image_ids[0]
    )
)

In [8]:
img1_npy

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

In [9]:
img1_png = Image.open(
        os.path.join(
            os.getenv('data_path'),'datasets/vindr_ds','images_png',study_id, image_ids[0][:-3]+'png'
    ))

In [10]:
from torchvision import transforms

In [11]:
transformer = transforms.ToTensor()

In [14]:
img1_png

'Portable network graphics'

In [15]:
tensor_png = transformer(img1_png)

In [16]:
tensor_npy = transformer(img1_npy.astype(np.int32))

In [17]:
(tensor_npy == tensor_png).sum()

tensor(9945240)

In [18]:
tensor_npy.shape.numel()

9945240

In [20]:
np.asarray(img1_png)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [32]:
img1_npy

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

In [33]:
np.savez_compressed(
            os.path.join(
                os.getenv('data_path'),'datasets/vindr_ds','images',study_id, image_ids[0][:-4]+'_v2.npz'
    )
    ,np.asarray(img1_png)
)

# Compare tensors with savez_compressed

In [ ]:
image_ids

['e2af0315474e547de5863a4468bb347d.npy',
 'a995ccf924881443381b6833b8828607.npy',
 'c7bcbb76529f8aa1d75cd72ffd7357e2.npy',
 'e2af0315474e547de5863a4468bb347d_v2.npy',
 '32325a132f79168bb761f3ae2a7c9969.npy']

In [34]:
img1_npy_compressed = np.load(
        os.path.join(
            os.getenv('data_path'),'datasets/vindr_ds','images',study_id, 'e2af0315474e547de5863a4468bb347d_v2.npz'
    )
)

In [42]:
np.testing.assert_array_equal(img1_npy, img1_npy_compressed['arr_0'])

In [43]:
tensor_png = transformer(img1_png)

In [44]:
tensor_npy = transformer(img1_npy_compressed['arr_0'])

In [45]:
(tensor_npy == tensor_png).sum()

tensor(9945240)

In [46]:
tensor_npy.shape.numel()

9945240